# Set the Working Directory

In [1]:
from sdm.utils import set_project_wd
set_project_wd()

Current Working Directory: /Users/matthewwhittle/Data Science/shefflied-bats


# Load Libraries

In [2]:
import elapid as ela
import geopandas as gpd
import pandas as pd
import xarray as xr
import rioxarray as rxr
from sdm.geo import generate_model_raster
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from elapid import MaxentModel, GeographicKFold, distance_weights
from sdm.maxent import prepare_occurence_data, filter_bats, extract_split, cv_maxent, train_maxent, filter_gdf_to_grid

# Grid

I only want to keep one point for each species in each 100m grid square. I'm going to do this by loading the model raster which is a null raster which all the predictors are modelled on. I'll convert it to a geodataframe and then take the inner spatial join between the nearest points with a threshold of 100m.

In [3]:
def generate_point_grid():
    model_raster = generate_model_raster()
    # Cooridinates represent the center of each pixel
    grid_df = model_raster.to_dataframe(name = "random").reset_index()
    grid_points = gpd.GeoDataFrame(grid_df, geometry = gpd.points_from_xy(grid_df.x, grid_df.y), crs = 27700)
    # Only keep the geometry column
    grid_points = grid_points[["geometry"]]
    return grid_points

grid_points = generate_point_grid()

grid_points.head()


/Users/matthewwhittle/opt/miniconda3/envs/bats/lib/python3.11/site-packages/pygeos/constructive.py:175: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(


,geometry
0,POINT (403914.879 426796.146)
1,POINT (404014.948 426796.146)
2,POINT (404115.017 426796.146)
3,POINT (404215.086 426796.146)
4,POINT (404315.154 426796.146)


# Bat Records & Background Points


In [4]:
bats = gpd.read_parquet('data/processed/sybg-bats.parquet')
bats.head()

,date,grid_reference,species_raw,activity_type,source_data,latin_name,common_name,genus,x,y,accuracy,geometry,grid_square_geom
0,2010-05-17 00:00:00,NZ115084,Common pipistrelle,Roost,"{""Recorder"":""BCT\/NE"",""Date"":1274054400000,""Gr...",Pipistrellus pipistrellus,Common Pipistrelle,Pipistrellus,411550.0,508450.0,100.0,POINT (411550.000 508450.000),"POLYGON ((411600.000 508400.000, 411600.000 50..."
1,2014-07-03 00:00:00,NZ14640021,Pipistrellus sp.,Unknown,"{""Recorder"":""Giles Manners"",""Date"":14043456000...",Pipistrellus sp.,Unidentified Pipistrelle,Pipistrellus,414645.0,500215.0,10.0,POINT (414645.000 500215.000),"POLYGON ((414650.000 500210.000, 414650.000 50..."
2,2013-11-28 00:00:00,NZ20291106,Soprano pipistrelle,Unknown,"{""Recorder"":""Natural England Volunteer Bat Roo...",Pipistrellus pygmaeus,Soprano Pipistrelle,Pipistrellus,420295.0,511065.0,10.0,POINT (420295.000 511065.000),"POLYGON ((420300.000 511060.000, 420300.000 51..."
3,2010-08-31 00:00:00,NZ170014,Unidentified bat species,Roost,"{""Recorder"":""BCT\/NE"",""Date"":1283212800000,""Gr...",Unknown,Unidentified Bat,Unknown,417050.0,501450.0,100.0,POINT (417050.000 501450.000),"POLYGON ((417100.000 501400.000, 417100.000 50..."
4,2009-05-20 00:00:00,NZ185116,Unidentified bat species,Roost,"{""Recorder"":""BCT\/NE"",""Date"":1242777600000,""Gr...",Unknown,Unidentified Bat,Unknown,418550.0,511650.0,100.0,POINT (418550.000 511650.000),"POLYGON ((418600.000 511600.000, 418600.000 51..."


In [5]:
latin_name = [
 'Pipistrellus pipistrellus',
 'Pipistrellus pygmaeus',
 'Plecotus auritus',
 'Myotis mystacinus',
 'Nyctalus noctula',
 'Myotis daubentonii',
 'Myotis nattereri',
 'Nyctalus leisleri',
 'Myotis brandtii',
 'Pipistrellus nathusii',
 'Eptesicus serotinus'
]

activity_type = [
    'Foraging',
    'Roost',
    None # Don't filters activity type
]

genus = [
    'Pipistrellus',
    'Plecotus',
    'Myotis',
    'Nyctalus',
    'Eptesicus'
]


In [6]:
bats = bats[bats.accuracy <= 100]


This function will be used once the data has been filtered to the right combination of species and behaviour to keep only the records which are unique to each 100m grid square.

In [7]:
background = gpd.read_parquet('data/processed/background-points.parquet')
background = background[["geometry"]]
background.head()

,geometry
363752,POINT (472223.236 416517.648)
83313,POINT (429711.807 381022.722)
171177,POINT (436034.552 392150.867)
324861,POINT (459022.462 411539.652)
281490,POINT (472438.437 406093.802)


I'm going to build a series of models based upon different taxonomic and behaviour classifications. I will build a model for:
- Each species and any behaviour type
- Each species and roosting or foraging
- Each genus and any behaviour type
- Each genus and roosting or foraging

# Point Annotation

In [8]:
from pathlib import Path

def load_evs(ev_folder:Path):
    # list the tifs
    ev_tifs = list(ev_folder.glob("*.tif"))

    #path = ev_tifs[0]
    def load_dataset(path):
        data = rxr.open_rasterio(path)
        # Extract the band name
        long_name = data.attrs["long_name"]
        if type(long_name) == str:
            long_name = [long_name]
        else:
            long_name = list(long_name)
        
        # prefix the band name with the file name
        long_name = [f"{path.stem}_{name}" for name in long_name]

        # Rename the band dimension and convert to a dataset
        data.coords["band"] = long_name
        return data.to_dataset(dim="band")

    evs = [load_dataset(path) for path in ev_tifs]

    evs = xr.merge(evs)

    return evs
ev_folder = Path("data/evs/")
evs = load_evs(ev_folder)
evs

<xarray.Dataset>
Dimensions:                                         (y: 563, x: 792)
Coordinates:
  * x                                               (x) float64 4.039e+05 ......
  * y                                               (y) float64 4.268e+05 ......
    spatial_ref                                     int64 0
Data variables: (12/107)
    climate_stats_temp_ann_var                      (y, x) float64 ...
    climate_stats_temp_ann_avg                      (y, x) float64 ...
    climate_stats_temp_mat_avg                      (y, x) float64 ...
    climate_stats_prec_ann_var                      (y, x) float64 ...
    climate_stats_prec_ann_avg                      (y, x) float64 ...
    climate_stats_wind_ann_var                      (y, x) float64 ...
    ...                                              ...
    ceh-land-cover-100m_Littoral rock               (y, x) float64 ...
    ceh-land-cover-100m_Littoral sediment           (y, x) float64 ...
    ceh-land-cover-100m_Saltmarsh                   (y, x) float64 ...
    ceh-land-cover-100m_Urban                       (y, x) float64 ...
    ceh-land-cover-100m_Suburban                    (y, x) float64 ...
    dtm-100m_LIDAR_Composite_2m_DTM_2022_Elevation  (y, x) float32 ...
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0
    long_name:      ('temp_ann_var', 'temp_ann_avg', 'temp_mat_avg', 'prec_an...

Many of the environmental variables are highly correlated with each other so I will only use a subset in the modelling

In [9]:
ev_subset_df = pd.read_csv("data/evs/ev_clusters_selected.csv")
keep_evs = ev_subset_df[ev_subset_df.include == 1].column_name.tolist()

# Subset the xr dataset to  only include the variables with the keep evs names
evs_to_model = evs[keep_evs]
evs_to_model

<xarray.Dataset>
Dimensions:                                         (y: 563, x: 792)
Coordinates:
  * x                                               (x) float64 4.039e+05 ......
  * y                                               (y) float64 4.268e+05 ......
    spatial_ref                                     int64 0
Data variables: (12/31)
    climate_stats_wind_ann_avg                      (y, x) float64 ...
    dtm-100m_LIDAR_Composite_2m_DTM_2022_Elevation  (y, x) float32 ...
    climate_stats_temp_ann_avg                      (y, x) float64 ...
    climate_stats_prec_ann_avg                      (y, x) float64 ...
    bioclim_bioclim_15                              (y, x) float32 ...
    os-distance-to-feature_distance_to_buildings    (y, x) float64 ...
    ...                                              ...
    ceh-land-cover-100m_Calcareous grassland        (y, x) float64 ...
    os-distance-to-feature_distance_to_major_roads  (y, x) float64 ...
    ceh-land-cover-100m_Urban                       (y, x) float64 ...
    os-distance-to-feature_distance_to_water        (y, x) float64 ...
    ceh-land-cover-100m_Improved grassland          (y, x) float64 ...
    ceh-land-cover-100m_Coniferous woodland         (y, x) float64 ...
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0
    long_name:      ('temp_ann_var', 'temp_ann_avg', 'temp_mat_avg', 'prec_an...

In [10]:
## Annotate points
from tempfile import NamedTemporaryFile
temp_ev_raster =  NamedTemporaryFile(suffix = ".tif", delete=False)
# Write the EVs to a temporary file and annotate the background and presence points

evs_to_model.rio.to_raster(temp_ev_raster.name)


ev_columns = list(evs_to_model.data_vars.keys())
bats_ant = ela.annotate(
    bats, 
    temp_ev_raster.name, 
    labels = ev_columns,
)
background = ela.annotate(
    background, 
    temp_ev_raster.name, 
    labels = ev_columns,
)

Raster:   0%|                              | 0/1 [00:00<?, ?it/s]

Sample:   0%|                              | 0/9316 [00:00<?, ?it/s]

Raster:   0%|                              | 0/1 [00:00<?, ?it/s]

Sample:   0%|                              | 0/10000 [00:00<?, ?it/s]

In [11]:
bats_ant.head()

,date,grid_reference,species_raw,activity_type,source_data,latin_name,common_name,genus,x,y,...,ceh-land-cover-100m_Heather grassland,ceh-land-cover-100m_Inland rock,ceh-land-cover-100m_Suburban,ceh-land-cover-100m_Neutral grassland,ceh-land-cover-100m_Calcareous grassland,os-distance-to-feature_distance_to_major_roads,ceh-land-cover-100m_Urban,os-distance-to-feature_distance_to_water,ceh-land-cover-100m_Improved grassland,ceh-land-cover-100m_Coniferous woodland
0,2010-05-17 00:00:00,NZ115084,Common pipistrelle,Roost,"{""Recorder"":""BCT\/NE"",""Date"":1274054400000,""Gr...",Pipistrellus pipistrellus,Common Pipistrelle,Pipistrellus,411550.0,508450.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2014-07-03 00:00:00,NZ14640021,Pipistrellus sp.,Unknown,"{""Recorder"":""Giles Manners"",""Date"":14043456000...",Pipistrellus sp.,Unidentified Pipistrelle,Pipistrellus,414645.0,500215.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2013-11-28 00:00:00,NZ20291106,Soprano pipistrelle,Unknown,"{""Recorder"":""Natural England Volunteer Bat Roo...",Pipistrellus pygmaeus,Soprano Pipistrelle,Pipistrellus,420295.0,511065.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2010-08-31 00:00:00,NZ170014,Unidentified bat species,Roost,"{""Recorder"":""BCT\/NE"",""Date"":1283212800000,""Gr...",Unknown,Unidentified Bat,Unknown,417050.0,501450.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2009-05-20 00:00:00,NZ185116,Unidentified bat species,Roost,"{""Recorder"":""BCT\/NE"",""Date"":1242777600000,""Gr...",Unknown,Unidentified Bat,Unknown,418550.0,511650.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Modelling

Modelling process:
1. Filter to the species and behaviour type
2. Get the unique points in each 100m grid square
3. Calculate the distance weights for the points
4. Define the model and fit using Cross Validation
5. Save the model

In [12]:
ela.MaxentModel().get_params()

{'beta_categorical': 1.0,
 'beta_hinge': 1.0,
 'beta_lqp': 1.0,
 'beta_multiplier': 1.5,
 'beta_threshold': 1.0,
 'clamp': True,
 'class_weights': 100,
 'convergence_tolerance': 2e-06,
 'feature_types': ['linear', 'hinge', 'product'],
 'n_cpus': 10,
 'n_hinge_features': 10,
 'n_lambdas': 100,
 'n_threshold_features': 10,
 'scorer': 'roc_auc',
 'tau': 0.5,
 'transform': 'cloglog',
 'use_lambdas': 'best',
 'use_sklearn': True}

In [13]:
def maxent_model(n_jobs = 1) -> Pipeline:

    model = Pipeline(
        [
        #   ("imputer", SimpleImputer(strategy="median")),
            ("scaler", StandardScaler()),
            ("maxent", MaxentModel(
                feature_types = ["linear", "product", "threshold"],
                beta_multiplier = 1,
                n_cpus = n_jobs,
            )),
        ]
    )
    return model

In [14]:
# Generate every combination of latin name and activity type
from itertools import product

# Generate all combinations of latin name and activity type
filter_combinations = list(product(latin_name, activity_type))

In [15]:
training_data = []
for latin_name, activity_type in filter_combinations:
    presence = filter_bats(bats_ant, latin_name=latin_name, activity_type=activity_type)

    if len(presence) < 15:
        continue

    occurrence = prepare_occurence_data(
        presence, background, grid_points, input_vars=ev_columns
    )
    training_data.append({
        "latin_name": latin_name,
        "activity_type": activity_type,
        "occurrence": occurrence,
    })

In [16]:
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import as_completed
from sdm.maxent import eval_train_model
import pandas as pd
from tqdm import tqdm

# Get the number of cpus
import multiprocessing
num_cpus = multiprocessing.cpu_count()

executor = ProcessPoolExecutor(num_cpus - 1)

# Submit tasks to the executor
futures = [executor.submit(eval_train_model, data["occurrence"], maxent_model()) for data in training_data]

# Collect results as they complete
results = []
for future in tqdm(as_completed(futures), total=len(futures)):
    result = future.result()
    if result is not None:
        results.append(result)


100%|██████████| 25/25 [03:28<00:00,  8.35s/it]


In [17]:
import numpy as np

# Convert the inputs and outputs to a dataframe
modelling_df = pd.DataFrame(
    [
        {
            "final_model": final_model,
            "cv_models": cv_models,
            "cv_scores": np.array(cv_scores),
        }
        for final_model, cv_models, cv_scores in results
    ]
)
inputs_df = pd.DataFrame(training_data)
# Combin them
results_df = pd.concat([inputs_df, modelling_df], axis=1)

# Mutate some columns
def count_presence(occurrence):
    return (occurrence["class"] == 1).sum()

results_df["n_presence"] = results_df.occurrence.apply(count_presence)
results_df["mean_cv_score"] = results_df.cv_scores.apply(np.mean)
results_df["mean_cv_score"] = results_df["mean_cv_score"].round(3)
results_df["std_cv_score"] = results_df.cv_scores.apply(np.std)
results_df["std_cv_score"] = results_df["std_cv_score"].round(3)

results_df["folds"] = results_df.cv_scores.apply(len)


results_df["activity_type"] = results_df.activity_type.fillna("All")

results_df.head()

,latin_name,activity_type,occurrence,final_model,cv_models,cv_scores,n_presence,mean_cv_score,std_cv_score,folds
0,Pipistrellus pipistrellus,Foraging,climate_stats_wind_ann_avg \ 0 ...,"(StandardScaler(), MaxentModel(beta_multiplier...","[(StandardScaler(), MaxentModel(beta_multiplie...","[0.4132308422194108, 0.5108735802013112, 0.900...",739,0.608,0.210,3
1,Pipistrellus pipistrellus,Roost,climate_stats_wind_ann_avg \ 0 ...,"(StandardScaler(), MaxentModel(beta_multiplier...","[(StandardScaler(), MaxentModel(beta_multiplie...","[0.6722718415069828, 0.6547586310342242, 0.648...",186,0.658,0.010,3
2,Pipistrellus pipistrellus,All,climate_stats_wind_ann_avg \ 0 ...,"(StandardScaler(), MaxentModel(beta_multiplier...","[(StandardScaler(), MaxentModel(beta_multiplie...","[0.6434966473636087, 0.7582150532260483, 0.628...",1038,0.677,0.058,3
3,Pipistrellus pygmaeus,Foraging,climate_stats_wind_ann_avg \ 0 ...,"(StandardScaler(), MaxentModel(beta_multiplier...","[(StandardScaler(), MaxentModel(beta_multiplie...","[0.6213036234902124, 0.811411992263056, 0.7951...",133,0.743,0.086,3
4,Pipistrellus pygmaeus,Roost,climate_stats_wind_ann_avg \ 0 ...,"(StandardScaler(), MaxentModel(beta_multiplier...","[(StandardScaler(), MaxentModel(beta_multiplie...","[0.7120355411954766, 0.7087195828505215, 0.571...",43,0.664,0.065,3


In [18]:
results_df

,latin_name,activity_type,occurrence,final_model,cv_models,cv_scores,n_presence,mean_cv_score,std_cv_score,folds
0,Pipistrellus pipistrellus,Foraging,climate_stats_wind_ann_avg \ 0 ...,"(StandardScaler(), MaxentModel(beta_multiplier...","[(StandardScaler(), MaxentModel(beta_multiplie...","[0.4132308422194108, 0.5108735802013112, 0.900...",739,0.608,0.210,3
1,Pipistrellus pipistrellus,Roost,climate_stats_wind_ann_avg \ 0 ...,"(StandardScaler(), MaxentModel(beta_multiplier...","[(StandardScaler(), MaxentModel(beta_multiplie...","[0.6722718415069828, 0.6547586310342242, 0.648...",186,0.658,0.010,3
2,Pipistrellus pipistrellus,All,climate_stats_wind_ann_avg \ 0 ...,"(StandardScaler(), MaxentModel(beta_multiplier...","[(StandardScaler(), MaxentModel(beta_multiplie...","[0.6434966473636087, 0.7582150532260483, 0.628...",1038,0.677,0.058,3
3,Pipistrellus pygmaeus,Foraging,climate_stats_wind_ann_avg \ 0 ...,"(StandardScaler(), MaxentModel(beta_multiplier...","[(StandardScaler(), MaxentModel(beta_multiplie...","[0.6213036234902124, 0.811411992263056, 0.7951...",133,0.743,0.086,3
4,Pipistrellus pygmaeus,Roost,climate_stats_wind_ann_avg \ 0 ...,"(StandardScaler(), MaxentModel(beta_multiplier...","[(StandardScaler(), MaxentModel(beta_multiplie...","[0.7120355411954766, 0.7087195828505215, 0.571...",43,0.664,0.065,3
5,Pipistrellus pygmaeus,All,climate_stats_wind_ann_avg \ 0 ...,"(StandardScaler(), MaxentModel(beta_multiplier...","[(StandardScaler(), MaxentModel(beta_multiplie...","[0.6761698780967361, 0.8619515724073364, 0.794...",201,0.778,0.077,3
6,Plecotus auritus,Foraging,climate_stats_wind_ann_avg \ 0 ...,"(StandardScaler(), MaxentModel(beta_multiplier...","[(StandardScaler(), MaxentModel(beta_multiplie...","[0.6248408801600291, 0.6879815100154083, 0.664...",43,0.659,0.026,3
7,Plecotus auritus,Roost,climate_stats_wind_ann_avg \ 0 ...,"(StandardScaler(), MaxentModel(beta_multiplier...","[(StandardScaler(), MaxentModel(beta_multiplie...","[0.44016171919271174, 0.6234944325590014, 0.58...",86,0.550,0.079,3
8,Plecotus auritus,All,climate_stats_wind_ann_avg \ 0 ...,"(StandardScaler(), MaxentModel(beta_multiplier...","[(StandardScaler(), MaxentModel(beta_multiplie...","[0.7581325275124899, 0.65321312212111, 0.68591...",142,0.699,0.044,3
9,Myotis mystacinus,Foraging,climate_stats_wind_ann_avg \ 0 ...,"(StandardScaler(), MaxentModel(beta_multiplier...","[(StandardScaler(), MaxentModel(beta_multiplie...","[0.6880220343325647, 0.761693495615757, 0.6620...",16,0.704,0.042,3


# Make Predictions

In [19]:
# Iterate through the df and make predictions using the best model
# save each prediction to a tif named after the latin name and activity type

for _, row in results_df.iterrows():
    latin_name = row.latin_name
    activity_type = row.activity_type
    model = row.final_model

    path_predict = Path(f"data/sdm_predictions/{latin_name}_{activity_type}.tif")
    ela.apply_model_to_rasters(
        model, 
        temp_ev_raster.name,
        output_path = path_predict,
        windowed = False,
    )

    # Add the prediction_path to the results_df
    results_df.loc[_, "prediction_path"] = path_predict

Window:   0%|                              | 0/1 [00:00<?, ?it/s]

Window:   0%|                              | 0/1 [00:00<?, ?it/s]

Window:   0%|                              | 0/1 [00:00<?, ?it/s]

Window:   0%|                              | 0/1 [00:00<?, ?it/s]

Window:   0%|                              | 0/1 [00:00<?, ?it/s]

...

Window:   0%|                              | 0/1 [00:00<?, ?it/s]

Window:   0%|                              | 0/1 [00:00<?, ?it/s]

Window:   0%|                              | 0/1 [00:00<?, ?it/s]

Window:   0%|                              | 0/1 [00:00<?, ?it/s]

Window:   0%|                              | 0/1 [00:00<?, ?it/s]

In [20]:
results_df.to_csv("data/sdm_predictions/results.csv", index=False)

In [ ]:
import leafmap
selected_path = results_df.prediction_path[0]
print(f"Mapping predictions for {selected_path.stem}")
m = leafmap.Map()
m.add_basemap("HYBRID")
m.add_raster(str(selected_path), layer_name = "Predictions", cmap = "viridis")

m

Mapping predictions for Pipistrellus pipistrellus_Roost


Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…